In [2]:
# 전체 재무재표를 가져오는 코드 
import sys 
sys.path.append('/var/www/bin/py/lib')
import pandas as pd
import itertools
import naver_finstate_detail as naver
import naver_acc_codelist
import sqlalchemy as sql
import numpy as np

In [3]:
def find_value(df, acc_code, acc_name, n):
    try:
        return df.loc[(acc_code, acc_name), n]
    except:
        return np.nan

def fin_export(df, n, rpt_key, acc_codes_list):
    acc_codes = acc_codes_list[rpt_key[1]]
    fin_dict = {}
    for acc_code in acc_codes:
        fin_dict[acc_codes[acc_code]] = find_value(df, acc_code, acc_codes[acc_code], n)
    return fin_dict


#읽어올 회계 코드를 가져옴 
acc_codes_list = naver_acc_codelist.account_list()

#index list를 만듬 
index_list = ['종목코드', '종목명', '구분', '회계', '년도', '분기']
for index_group in acc_codes_list.keys():
    for index in acc_codes_list[index_group].values():
        index_list.append(index)

# 종목 리스트를 가져옴 
connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
krx_list = pd.read_sql_query(query, sql_engine)

frq_list = [('0', '연간'), ('1', '분기') ] 
rpt_list = [('0', '손익계산서'), ('1', '재무상태표'), ('2', '현금흐름표')]
finGubun_list = [('IFRSL', 'IFRS연결(기본값)'), ('IFRSS', 'KIFRS별도')]

# 종목 별 재무제표를 가져옴
total_list = []

while True:
    try:
        for ix, row in krx_list.iterrows():
            stock_dict = {'종목코드': row['종목코드'], '종목명': row['종목명']}
            for frq, finGubun in itertools.product(frq_list,  finGubun_list):
                data_dict = {(frq[1], finGubun[1].replace('(기본값)', '')): {'손익계산서': {}, '재무상태표': {}, '현금흐름표': {}}}
                for rpt in rpt_list: 
                    df = naver.naver_finstate_detail(row['종목코드'], rpt=rpt[0], frq=frq[0], finGubun=finGubun[0])
                    #print([row['Symbol'], row['Name']])
                    for date in df.columns:
                        if frq[0] == '0':
                            data_dict[(frq[1], finGubun[1].replace('(기본값)', ''))][rpt[1]][(date.year,)] = fin_export(df, date, rpt, acc_codes_list)
                        if frq[0] == '1':
                            key = (date.year, date.quarter)
                            data_dict[(frq[1], finGubun[1].replace('(기본값)', ''))][rpt[1]][key] = fin_export(df, date, rpt, acc_codes_list)
                        #print(data_dict)
                stock_dict.update(data_dict)
            first_key = list(stock_dict)[2:]
            for key1 in first_key: #first_key ('연간', 'IFRS연결'), ('연간', 'KIFRS별도'), ('분기', 'IFRS연결'), ('분기', 'KIFRS별도')
                for key2 in stock_dict[key1]['손익계산서'].keys(): #key2 날짜를 선택 
                    if len(key2) == 1:
                        year = key2[0]
                        quarter = np.nan
                    elif len(key2) == 2:
                        year = key2[0]
                        quarter = key2[1]
                    result_list = [stock_dict['종목코드'], stock_dict['종목명'], key1[0], key1[1], year, quarter]
                    for i in stock_dict[key1]['손익계산서'][key2].values():
                        result_list.append(i)
                    for i in stock_dict[key1]['재무상태표'][key2].values():
                        result_list.append(i)
                    for i in stock_dict[key1]['현금흐름표'][key2].values():
                        result_list.append(i)
                    total_list.append(result_list)
    except Exception as e:
        print([ix, row['종목명'], row['종목코드']])
        print(e)
        print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
        krx_list = krx_list[ix:]
        krx_list = krx_list.reset_index(drop = True)
        pass
    else:
        break
        
df_all = pd.DataFrame(total_list, columns = index_list)
df_all.insert(loc=0, column='No', value=np.nan)
df_all.insert(loc=7, column='컨센서스여부', value='실적') 

df_all

,No,종목코드,종목명,구분,회계,년도,분기,컨센서스여부,매출액(수익),영업이익,...,유동성장기부채,매출채권,발행주식수,보통주,우선주,유형자산감가상각비,기타무형자산상각비,영업활동으로인한현금흐름,투자활동으로인한현금흐름,재무활동으로인한현금흐름
0,1,000020,동화약품,연간,IFRS연결,2015.0,NaN,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,000020,동화약품,연간,IFRS연결,2016.0,NaN,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,000020,동화약품,연간,IFRS연결,2017.0,NaN,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,000020,동화약품,연간,IFRS연결,2018.0,NaN,실적,3066.025890,112.257800,...,NaN,733.460870,27931.0,27931.0,NaN,103.416588,10.155212,-43.600777,172.173888,-38.311600
4,5,000020,동화약품,연간,IFRS연결,2019.0,NaN,실적,3071.500258,99.177463,...,NaN,706.247730,27931.0,27931.0,NaN,104.401481,10.445310,132.849662,-111.784451,-49.375742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47269,47270,122830,원포유,분기,KIFRS별도,2018.0,4.0,실적,NaN,NaN,...,NaN,2.178101,6550.0,6550.0,NaN,NaN,NaN,NaN,NaN,NaN
47270,47271,122830,원포유,분기,KIFRS별도,2019.0,1.0,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47271,47272,122830,원포유,분기,KIFRS별도,2019.0,2.0,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47272,47273,122830,원포유,분기,KIFRS별도,2019.0,3.0,실적,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4)
}

for i in list(df_all.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 

df_year = df_all.loc[df_all['구분'] == '연간']
df_quarter = df_all.loc[df_all['구분'] == '분기']

No_list = []
for i in range(0, len(df_year)):
    No_list.append(i+1)
df_year['No'] = No_list

No_list = []
for i in range(0, len(df_quarter)):
    No_list.append(i+1)
df_quarter['No'] = No_list


df_year.to_sql(name="네이버재무연간_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `네이버재무연간_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `네이버재무연간_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `네이버재무연간_rawdata` ADD INDEX(`종목코드`);')
    
    
df_quarter.to_sql(name="네이버재무분기_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `네이버재무분기_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `네이버재무분기_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `네이버재무분기_rawdata` ADD INDEX(`종목코드`);')

<ipython-input-6-997cdb9974f6>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year['No'] = No_list
<ipython-input-6-997cdb9974f6>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quarter['No'] = No_list


In [7]:
# 끝나고 지우기

import pandas as pd
import sqlalchemy as sql

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT * FROM `네이버재무분기_rawdata`"
df = pd.read_sql_query(query, sql_engine)

df_copy = df.copy()

df_copy['GP/A'] = (df_copy['매출총이익']/df_copy['자산총계']) * 100
df_copy['영업이익율'] = (df_copy['영업이익']/df_copy['매출액(수익)']) * 100
df_copy['ROE'] = (df_copy['(지배주주지분)당기순이익']/df_copy['지배주주지분']) * 100
df_copy['ROA'] = (df_copy['(지배주주지분)당기순이익']/df_copy['자산총계']) * 100
df_copy['CFO'] = df_copy['영업활동으로인한현금흐름']/df_copy['자산총계']
df_copy['당좌비율'] = (df_copy['유동자산']-df_copy['재고자산'])*100/df_copy['유동부채']
df_copy['유동비율'] = (df_copy['유동자산']/df_copy['유동부채']) * 100
df_copy['부채비율'] = (df_copy['부채총계']/df_copy['지배주주지분']) * 100
df_copy['매출채권 회수기간'] = 365/(df_copy['매출액(수익)']/df_copy['매출채권'])
df_copy['재고자산 회수 기간'] = 365/(df_copy['매출원가']/df_copy['재고자산'])
df_copy['FCFF'] = df_copy['영업활동으로인한현금흐름']-df_copy['유형자산']
df_copy['매출총이익률'] = (df_copy['매출총이익']/df_copy['매출액(수익)']) * 100
df_copy['총자산회전율'] = (df_copy['매출액(수익)']/df_copy['자산총계']) * 100
df_copy['차입금비율'] = ((df_copy['장기차입금']+df_copy['단기차입금'] + df_copy['사채'] + df_copy['유동성장기부채'])/df_copy['지배주주지분']) * 100

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4)
}

for i in list(df_copy.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 
    
df_copy.to_sql(name="네이버재무분기_가치지표추가", con=sql_engine, if_exists='replace', index=False, 
         dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `네이버재무분기_가치지표추가` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `네이버재무분기_가치지표추가` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `네이버재무분기_가치지표추가` ADD INDEX(`종목코드`);')
    
import pandas as pd
import sqlalchemy as sql

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')
query = "SELECT * FROM `네이버재무연간_rawdata`"
df = pd.read_sql_query(query, sql_engine)

df_copy = df.copy()

df_copy['GP/A'] = (df_copy['매출총이익']/df_copy['자산총계']) * 100
df_copy['영업이익율'] = (df_copy['영업이익']/df_copy['매출액(수익)']) * 100
df_copy['ROE'] = (df_copy['(지배주주지분)당기순이익']/df_copy['지배주주지분']) * 100
df_copy['ROA'] = (df_copy['(지배주주지분)당기순이익']/df_copy['자산총계']) * 100
df_copy['CFO'] = df_copy['영업활동으로인한현금흐름']/df_copy['자산총계']
df_copy['당좌비율'] = (df_copy['유동자산']-df_copy['재고자산'])*100/df_copy['유동부채']
df_copy['유동비율'] = (df_copy['유동자산']/df_copy['유동부채']) * 100
df_copy['부채비율'] = (df_copy['부채총계']/df_copy['지배주주지분']) * 100
df_copy['매출채권 회수기간'] = 365/(df_copy['매출액(수익)']/df_copy['매출채권'])
df_copy['재고자산 회수 기간'] = 365/(df_copy['매출원가']/df_copy['재고자산'])
df_copy['FCFF'] = df_copy['영업활동으로인한현금흐름']-df_copy['유형자산']
df_copy['매출총이익률'] = (df_copy['매출총이익']/df_copy['매출액(수익)']) * 100
df_copy['총자산회전율'] = (df_copy['매출액(수익)']/df_copy['자산총계']) * 100
df_copy['차입금비율'] = ((df_copy['장기차입금']+df_copy['단기차입금'] + df_copy['사채'] + df_copy['유동성장기부채'])/df_copy['지배주주지분']) * 100

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4)
}

for i in list(df_copy.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 
    
df_copy.to_sql(name="네이버재무연간_가치지표추가", con=sql_engine, if_exists='replace', index=False, 
         dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `네이버재무연간_가치지표추가` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `네이버재무연간_가치지표추가` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `네이버재무연간_가치지표추가` ADD INDEX(`종목코드`);')
    
#사실 안지워도되고 

In [ ]:
#3분기 미반영 데이터 가져오는 코드 

In [ ]:
import sys 
sys.path.append('/var/www/bin/py/lib')
import pandas as pd
import itertools
import naver_finstate_detail as naver
import naver_acc_codelist
import sqlalchemy as sql
import numpy as np
import time
from datetime import datetime, timedelta

def find_value(df, acc_code, acc_name, n):
    try:
        return df.loc[[(acc_code, acc_name)], [n]].values[0][0]
    except:
        return np.nan

def fin_export(df, n, rpt_key, acc_codes_list):
    acc_codes = acc_codes_list[rpt_key[1]]
    fin_dict = {}
    for acc_code in acc_codes:
        fin_dict[acc_codes[acc_code]] = find_value(df, acc_code, acc_codes[acc_code], n)
    return fin_dict

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')

# 새로운 분기 30일 이후부터 크롤링을 시작합니다. 
current_time = datetime.now()
time = pd.to_datetime(current_time)
현재년도 = time.year
현재분기 = time.quarter

if 현재분기 == 1:
    target_year = time.year-1
    target_quarter = 4
else:
    target_year = 현재년도
    target_quarter = 현재분기-1

과거시간_1개원전 = current_time - timedelta(days=30)
과거분기_1개월전 = pd.to_datetime(과거시간_1개원전).quarter

if 과거분기_1개월전 != 현재분기:
    sys.exit()

# 최신 분기 자료가 올라오지 않은 기업 리스트를 뽑아냄 
query = "SELECT DISTINCT `종목코드` FROM `네이버재무분기_rawdata` WHERE `년도` = {} AND `분기` = {}".format(target_year, target_quarter)
df = pd.read_sql_query(query, sql_engine)
complete_list = list(df['종목코드'])

company_list = []
query = "SELECT `종목코드`, `종목명` FROM `종목마스터`"
krx_list = pd.read_sql_query(query, sql_engine)

for i in range(0, len(krx_list)):
    if not krx_list.iloc[i]['종목코드'] in complete_list:
        company_list.append([krx_list.iloc[i]['종목코드'], krx_list.iloc[i]['종목명']])
분기미반영_DF = pd.DataFrame(company_list, columns= ['종목코드', '종목명'])

#4분기 재무제표를 불러올 때는 최근 연간 자료가 올라오지 않은 기업 리스트도 뽑아냄 
if quarter == 4:
    query = "SELECT DISTINCT `종목코드` FROM `네이버재무연간_rawdata` WHERE `년도` = {}".format(target_year)
    df = pd.read_sql_query(query, sql_engine)
    complete_list = list(df['종목코드'])
    
    company_list = []
    for i in range(0, len(krx_list)):
        if not krx_list.iloc[i]['종목코드'] in complete_list:
            company_list.append([krx_list.iloc[i]['종목코드'], krx_list.iloc[i]['종목명']])
    연간미반영_DF = pd.DataFrame(company_list, columns= ['종목코드', '종목명'])
    
#읽어올 회계 코드를 가져옴 
acc_codes_list = naver_acc_codelist.account_list()

#index list를 만듬 
index_list = ['종목코드', '종목명', '구분', '회계', '년도', '분기']
for index_group in acc_codes_list.keys():
    for index in acc_codes_list[index_group].values():
        index_list.append(index)

frq_list = [('1', '분기') ] 
rpt_list = [('0', '손익계산서'), ('1', '재무상태표'), ('2', '현금흐름표')]
finGubun_list = [('IFRSL', 'IFRS연결(기본값)'), ('IFRSS', 'KIFRS별도')]

# 종목 별 분기 재무제표를 가져옴
total_list = []

while True:
    try:
        for ix, row in 분기미반영_DF.iterrows():
            stock_dict = {'종목코드': row['종목코드'], '종목명': row['종목명']}
            for frq, finGubun in itertools.product(frq_list,  finGubun_list):
                data_dict = {(frq[1], finGubun[1].replace('(기본값)', '')): {'손익계산서': {}, '재무상태표': {}, '현금흐름표': {}}}
                for rpt in rpt_list: 
                    df = naver.naver_finstate_detail(row['종목코드'], rpt=rpt[0], frq=frq[0], finGubun=finGubun[0])
                    for date in df.columns:
                        if (date.year, date.quarter) == (target_year, target_quarter):
                            key = (date.year, date.quarter)
                            data_dict[(frq[1], finGubun[1].replace('(기본값)', ''))][rpt[1]][key] = fin_export(df, date, rpt, acc_codes_list)
                stock_dict.update(data_dict)
            first_key = list(stock_dict)[2:]
            for key1 in first_key: #first_key ('연간', 'IFRS연결'), ('연간', 'KIFRS별도'), ('분기', 'IFRS연결'), ('분기', 'KIFRS별도')
                for key2 in stock_dict[key1]['손익계산서'].keys(): #key2 날짜를 선택 
                    if len(key2) == 1:
                        year = key2[0]
                        quarter = np.nan
                    elif len(key2) == 2:
                        year = key2[0]
                        quarter = key2[1]
                    result_list = [stock_dict['종목코드'], stock_dict['종목명'], key1[0], key1[1], year, quarter]
                    for i in stock_dict[key1]['손익계산서'][key2].values():
                        result_list.append(i)
                    for i in stock_dict[key1]['재무상태표'][key2].values():
                        result_list.append(i)
                    for i in stock_dict[key1]['현금흐름표'][key2].values():
                        result_list.append(i)
                    total_list.append(result_list)
    except Exception as e:
        print([ix, row['종목명'], row['종목코드']])
        print(e)
        print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
        분기미반영_DF = 분기미반영_DF[ix:]
        분기미반영_DF = 분기미반영_DF.reset_index(drop = True)
        pass
    else:
        break

분기_df = pd.DataFrame(total_list, columns = index_list)
분기_df.insert(loc=0, column='No', value=np.nan)
분기_df.insert(loc=7, column='컨센서스여부', value='실적') 

query = "SELECT* FROM `네이버재무분기_rawdata`"
과거분기_df = pd.read_sql_query(query, sql_engine)
Total_분기데이터 = pd.concat([분기_df, 과거분기_df])

No_list = []
for i in range(0, len(Total_분기데이터)):
    No_list.append(i+1)
Total_분기데이터['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '구분': sql.types.VARCHAR(2),
    '회계': sql.types.VARCHAR(7),
    '년도': sql.types.INTEGER(),
    '분기': sql.types.INTEGER(), 
    '컨센서스여부': sql.types.VARCHAR(4)
}

for i in list(Total_분기데이터.columns)[8:]:
    dtype[i] = sql.types.FLOAT() 
    
Total_분기데이터.to_sql(name="네이버재무분기_rawdata", con=sql_engine, if_exists='replace', index=False, 
             dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `네이버재무분기_rawdata` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `네이버재무분기_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `네이버재무분기_rawdata` ADD INDEX(`종목코드`);')
    

# 연간 데이터 필요시 아래코드 
if target_quarter == 4:
    frq_list = [('0', '연간')] 
    rpt_list = [('0', '손익계산서'), ('1', '재무상태표'), ('2', '현금흐름표')]
    finGubun_list = [('IFRSL', 'IFRS연결(기본값)'), ('IFRSS', 'KIFRS별도')]
    
    total_list = []
    while True:
        try:
            for ix, row in 연간미반영_DF.iterrows():
                stock_dict = {'종목코드': row['종목코드'], '종목명': row['종목명']}
                for frq, finGubun in itertools.product(frq_list,  finGubun_list):
                    data_dict = {(frq[1], finGubun[1].replace('(기본값)', '')): {'손익계산서': {}, '재무상태표': {}, '현금흐름표': {}}}
                    for rpt in rpt_list: 
                        df = naver.naver_finstate_detail(row['종목코드'], rpt=rpt[0], frq=frq[0], finGubun=finGubun[0])
                        for date in df.columns:
                            if date.year == target_year:
                                data_dict[(frq[1], finGubun[1].replace('(기본값)', ''))][rpt[1]][(date.year,)] = fin_export(df, date, rpt, acc_codes_list)
                    stock_dict.update(data_dict)
                first_key = list(stock_dict)[2:]
                for key1 in first_key: #first_key ('연간', 'IFRS연결'), ('연간', 'KIFRS별도'), ('분기', 'IFRS연결'), ('분기', 'KIFRS별도')
                    for key2 in stock_dict[key1]['손익계산서'].keys(): #key2 날짜를 선택 
                        if len(key2) == 1:
                            year = key2[0]
                            quarter = np.nan
                        elif len(key2) == 2:
                            year = key2[0]
                            quarter = key2[1]
                        result_list = [stock_dict['종목코드'], stock_dict['종목명'], key1[0], key1[1], year, quarter]
                        for i in stock_dict[key1]['손익계산서'][key2].values():
                            result_list.append(i)
                        for i in stock_dict[key1]['재무상태표'][key2].values():
                            result_list.append(i)
                        for i in stock_dict[key1]['현금흐름표'][key2].values():
                            result_list.append(i)
                        total_list.append(result_list)
        except Exception as e:
            print([ix, row['종목명'], row['종목코드']])
            print(e)
            print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
            연간미반영_DF = 연간미반영_DF[ix:]
            연간미반영_DF = 연간미반영_DF.reset_index(drop = True)
            pass
        else:
            break

    연간_df = pd.DataFrame(total_list, columns = index_list)
    연간_df.insert(loc=0, column='No', value=np.nan)
    연간_df.insert(loc=7, column='컨센서스여부', value='실적') 

    query = "SELECT* FROM `네이버재무연간_rawdata`"
    과거연간_df = pd.read_sql_query(query, sql_engine)
    Total_연간데이터 = pd.concat([연간_df, 과거연간_df])

    No_list = []
    for i in range(0, len(Total_연간데이터)):
        No_list.append(i+1)
    Total_연간데이터['No'] = No_list

    dtype = {
       'No': sql.types.BIGINT(),
        '종목코드': sql.types.VARCHAR(6), 
        '종목명': sql.types.VARCHAR(50), 
        '구분': sql.types.VARCHAR(2),
        '회계': sql.types.VARCHAR(7),
        '년도': sql.types.INTEGER(),
        '분기': sql.types.INTEGER(), 
        '컨센서스여부': sql.types.VARCHAR(4)
    }

    for i in list(Total_연간데이터.columns)[8:]:
        dtype[i] = sql.types.FLOAT() 

    Total_연간데이터.to_sql(name="네이버재무연간_rawdata", con=sql_engine, if_exists='replace', index=False, 
                 dtype = dtype)

    with sql_engine.connect() as con:
        con.execute('ALTER TABLE `네이버재무연간_rawdata` ADD PRIMARY KEY(`No`);')
        con.execute('ALTER TABLE `네이버재무연간_rawdata` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
        con.execute('ALTER TABLE `네이버재무연간_rawdata` ADD INDEX(`종목코드`);')

# 최신 분기 자료가 올라오지 않은 기업 리스트를 뽑아냄 
query = "SELECT DISTINCT `종목코드` FROM `네이버재무분기_rawdata` WHERE `년도` = {} AND `분기` = {}".format(target_year, target_quarter)
complete_df = pd.read_sql_query(query, sql_engine)

query = "SELECT DISTINCT `종목코드` FROM `종목마스터`"
company_df = pd.read_sql_query(query, sql_engine)

left_company_number = len(company_df)-len(complete_df)

now = time.localtime()
current_time = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

if quarter ==4:
    print("[" + current_time + "] " + "네이버재무분기_rawdata, 네이버재무연간_rawdata 업데이트 완료되었습니다. " + str(len(분기_df) + len(연간_df))+"개의 재무 정보가 추가되었습니다. "+str(left_company_number)+"개 기업의 재무정보의 추가가 필요합니다.")
    
else:
    print("[" + current_time + "] " + "네이버재무분기_rawdata 업데이트 완료되었습니다. " + str(len(분기_df))+"개의 재무 정보가 추가되었습니다. "+str(left_company_number)+"개 기업의 재무정보의 추가가 필요합니다.")
